## Specification aggregator for bioschemas

The DDE intentionally does not allow existing namespaces to be used by others. This is a feature that ensures that user-generated schemas or user-customized schemas are not confused for existing, registered schemas. Unfortunately, this means that users looking to update an existing bioschemas cannot use the bioschemas namespace when creating a schema within the DDE. The aggregator includes a function will replace the temporary namespace with the bioschemas namespace for the merge.

### To do
1. Include a check for multiple same classes:
  * A profile might reference another profile. In order for the profile to work, a dummy class of the referenced profile may need to be created. This dummy profile should be included in the merged file ONLY IF the actual class (which should have a validation section of its own) does not exist
  * Use the existence of a validation to determine which class to keep in the event of the same class coming from different profiles (real or dummy)


2. Include a check for multiple same properties:
  * A single property might be used across different bioschemas classes
  * This means that the `"schema:domainIncludes"` property should be updated to reflect ALL of the classes that use it, rather than just the first class that uses it.
  * eg- "bioschemas:output" might be a property in "ComputationalTool" and "ComputationalWorkflow". Since these profiles are developed separately, the one in "ComputationalTool" will include `"schema:domainIncludes": {"@id": "bioschemas:ComputationalTool"}` while the one from ComputationalWorkflow will include `"schema:domainIncludes": {"@id": "bioschemas:ComputationalWorkflow"}`. These will need to be merged into a single property with `"schema:domainIncludes": [{"@id": "bioschemas:ComputationalTool"},{"@id": "bioschemas:ComputationalWorkflow"}]`

In [2]:
import json
import requests
from pandas import read_csv

In [3]:
def get_raw_url(url):
    if 'raw' not in url:
        rawrawurl = url.replace('github.com','raw.githubusercontent.com')
        rawurl = rawrawurl.replace('/blob/master/','/master/')
    else:
        rawurl = url
    return(rawurl)

In [ ]:
def rename_namespace(spec_list,eachurl,rawtext):
    tmpinfo = spec_list.loc[spec_list['url']==eachurl]
    tmpnamespace = tmpinfo.iloc[0]['namespace']
    if namespace!='bioschemas'
        tmptext = '"@id": "'+tmpnamespace+':'
        cleantext = rawtext.replace(tmptext,'"@id": "bioschemas:')
    else:
        cleantext = rawtext
    return(cleantext)

In [4]:
def merge_specs(spec_list):
    bioschemas_json = {}
    graphlist = []
    for eachurl in spec_list['url']:
        rawurl = get_raw_url(eachurl)
        r = requests.get(rawurl)
        if r.status_code == 200:
            cleantext = rename_namespace(spec_list,eachurl,r.text)
            spec_json = json.loads(cleantext)
            bioschemas_json['@context'] = spec_json['@context']
            for x in spec_json['@graph']:
                if x not in graphlist:
                    graphlist.append(x)
    bioschemas_json['@graph']=graphlist
    return(bioschemas_json)

In [5]:
def update_specs():
    spec_list = read_csv('specifications_list.txt',delimiter='\t',header=0)
    bioschemas_json = merge_specs(spec_list)
    with open('bioschemas.json','w') as outfile:
        json.dump(bioschemas_json,outfile)

In [19]:
## main
update_specs()
